In [4]:
# Import necessary packages
import os 
import folium
from folium import plugins
import rioxarray as rxr
import earthpy as et
import earthpy.spatial as es

# Import data from EarthPy
data = et.data.get_data('colorado-flood')

# Set working directory to earth-analytics
os.chdir(os.path.join(et.io.HOME, 'earth-analytics', 'data'))

ModuleNotFoundError: No module named 'earthpy'

In [5]:
# Create a map using the Map() function and the coordinates for Boulder, CO
m = folium.Map(location=[40.0150, -105.2705])

# Display the map
m

In [6]:
# Create a map using Stamen Terrain as the basemap
m = folium.Map(location=[40.0150, -105.2705],
               tiles = 'Stamen Terrain')

# Display map
m

In [7]:
# Create a map using Stamen Terrain, centered on Boulder, CO
m = folium.Map(location=[40.0150, -105.2705], 
               tiles = 'Stamen Terrain')

# Add marker for Boulder, CO
folium.Marker(
    location=[40.009515, -105.242714], # coordinates for the marker (Earth Lab at CU Boulder)
    popup='Earth Lab at CU Boulder', # pop-up label for the marker
    icon=folium.Icon()
).add_to(m)

# Display m
m

RasterioIOError: colorado-flood/spatial/boulder-leehill-rd/post-flood/lidar/post_DTM.tif: No such file or directory